In [2]:
import numpy as np
import pandas as pd

In [4]:
def validate_4bit_adder(adder_data, start_time=2.5e-9, interval=5e-9):
    print(f"{'Time':<12} {'Carry_in':<10} {'a':<15} {'b':<15} {'Sum (Actual)':<15} {'Carry_out (Actual)':<20} {'Sum (Expected)':<20} {'Carry_out (Expected)':<25}")
    print("-" * 122)
    issues = []
    eval_times = np.arange(start_time, adder_data['/c0 X'].max(), interval)

    for t in eval_times:
        prior_row = adder_data[adder_data['/c0 X'] <= t].iloc[-1]
        next_row = adder_data[adder_data['/c0 X'] > t].iloc[0]

        interpolated_vals = {}
        for col in adder_data.columns:
            if ' X' not in col:  # Skip the time-related columns
                t1, t2 = prior_row['/c0 X'], next_row['/c0 X']
                v1, v2 = prior_row[col], next_row[col]
                interpolated_vals[col] = v1 + (v2 - v1) * (t - t1) / (t2 - t1)

        carry_in = int(interpolated_vals['/c0 Y'] > 0.5)
        a_signals = [int(interpolated_vals[f'/a{i} Y'] > 0.5) for i in range(4)]
        b_signals = [int(interpolated_vals[f'/b{i} Y'] > 0.5) for i in range(4)]
        sum_signals = [int(interpolated_vals[f'/s{i} Y'] > 0.5) for i in range(4)]
        carry_out = int(interpolated_vals['/c4 Y'] > 0.5)

        generate = [a_signals[i] & b_signals[i] for i in range(4)]
        propagate = [a_signals[i] ^ b_signals[i] for i in range(4)]
        carries = [carry_in]  # Carry chain starting with carry-in

        for i in range(4):
            carries.append(generate[i] | (propagate[i] & carries[i]))

        expected_sums = [propagate[i] ^ carries[i] for i in range(4)]
        expected_carry_out = carries[4]

        print(f"{t:<12.2e} {carry_in:<10} {str(a_signals):<15} {str(b_signals):<15} {str(sum_signals):<15} {carry_out:<20} {str(expected_sums):<20} {expected_carry_out:<25}")

        for i in range(4):
            carries.append(generate[i] | (propagate[i] & carries[i]))

        for i in range(4):
            expected_sum = propagate[i] ^ carries[i]
            if sum_signals[i] != expected_sum:
                issues.append((t, f'/s{i}', sum_signals[i], expected_sum))
        if carries[4] != carry_out:
            issues.append((t, '/c4', carry_out, carries[4]))

    return issues

print("Validating 4-Bit Adder Logic and Printing States...")
try:
    adder_data = pd.read_csv("Simulations Data/4bit_addr.csv")
    errors = validate_4bit_adder(adder_data)

    if errors:
        print("Issues detected in 4-bit adder logic:")
        for error in errors:
            print(f"Time: {error[0]:.2e}, Signal: {error[1]}, Actual: {error[2]}, Expected: {error[3]}")
    else:
        print("4-bit adder logic verified successfully. No issues detected.")
except FileNotFoundError:
    print("Error: File 'flookahead_adder.csv' not found. Please check the file's location.")


Validating 4-Bit Adder Logic and Printing States...
Time         Carry_in   a               b               Sum (Actual)    Carry_out (Actual)   Sum (Expected)       Carry_out (Expected)     
--------------------------------------------------------------------------------------------------------------------------
2.50e-09     0          [0, 0, 0, 0]    [0, 0, 0, 0]    [0, 0, 0, 0]    0                    [0, 0, 0, 0]         0                        
7.50e-09     0          [0, 0, 0, 0]    [0, 0, 0, 0]    [0, 0, 0, 0]    0                    [0, 0, 0, 0]         0                        
1.25e-08     0          [0, 0, 0, 0]    [0, 0, 0, 0]    [0, 0, 0, 0]    0                    [0, 0, 0, 0]         0                        
1.75e-08     0          [1, 0, 0, 0]    [0, 1, 0, 1]    [1, 1, 0, 1]    0                    [1, 1, 0, 1]         0                        
2.25e-08     1          [1, 1, 1, 1]    [0, 1, 1, 1]    [0, 1, 1, 1]    1                    [0, 1, 1, 1]         1          